In [1]:
import numpy as np
import pandas as pd

from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder

import pickle

# Import dataset

In [2]:
data = pd.read_csv("../DataFormating/final.csv")

In [3]:
data.head()

,Away Team Goals,Away Team Name,Home Team Goals,Home Team Name,Year,home_rank,home_total_points,home_previous_points,home_rank_change,home_cur_year_avg,...,away_cur_year_avg,away_cur_year_avg_weighted,away_last_year_avg,away_last_year_avg_weighted,away_two_year_ago_avg,away_two_year_ago_weighted,away_three_year_ago_avg,away_three_year_ago_weighted,Home Avg Goals,Away Avg Goals
0,1.0,Croatia,3.0,Brazil,2014.0,10,1102.21,1102,0,668.33,...,430.28,430.28,525.55,262.77,568.32,170.50,535.83,107.17,0.0,0.0
1,0.0,Egypt,2.0,Senegal,2014.0,68,525.55,536,-3,282.37,...,468.13,468.13,219.01,109.51,223.39,67.02,518.35,103.67,0.0,0.0
2,2.0,Uruguay,0.0,Japan,2014.0,48,640.56,638,-1,208.37,...,625.97,625.97,303.35,151.68,726.15,217.84,681.79,136.36,0.0,0.0
3,0.0,Spain,1.0,France,2014.0,20,893.16,893,0,435.22,...,699.69,699.69,830.72,415.36,610.39,183.12,1044.29,208.86,0.0,0.0
4,0.0,Australia,2.0,Belgium,2014.0,11,1098.31,1098,0,652.93,...,237.59,237.59,246.99,123.49,442.97,132.89,387.21,77.44,0.0,0.0


# Set `X` and `y`

In [4]:
data.columns

Index(['Away Team Goals', 'Away Team Name', 'Home Team Goals',
       'Home Team Name', 'Year', 'home_rank', 'home_total_points',
       'home_previous_points', 'home_rank_change', 'home_cur_year_avg',
       'home_cur_year_avg_weighted', 'home_last_year_avg',
       'home_last_year_avg_weighted', 'home_two_year_ago_avg',
       'home_two_year_ago_weighted', 'home_three_year_ago_avg',
       'home_three_year_ago_weighted', 'away_rank', 'away_total_points',
       'away_previous_points', 'away_rank_change', 'away_cur_year_avg',
       'away_cur_year_avg_weighted', 'away_last_year_avg',
       'away_last_year_avg_weighted', 'away_two_year_ago_avg',
       'away_two_year_ago_weighted', 'away_three_year_ago_avg',
       'away_three_year_ago_weighted', 'Home Avg Goals', 'Away Avg Goals'],
      dtype='object')

In [5]:
X = data.drop(["Away Team Goals", "Home Team Goals"], axis=1)

y = []
for i in range(len(data)):
    home_team_goals = data.iloc[i]["Home Team Goals"]
    away_team_goals = data.iloc[i]["Away Team Goals"]
    
    if home_team_goals > away_team_goals:
        y.append(1)
    elif home_team_goals < away_team_goals:
        y.append(2)
    else:
        y.append(0)

In [6]:
# Test
assert len(X) == len(y)

### Encode textual features from the `X` dataset

In [7]:
word_cup_teams = [
    "Egypt",
    "Morocco",
    "Nigeria",
    "Senegal",
    "Tunisia",
    "Australia",
    "IR Iran",
    "Japan",
    "Korea DPR",
    "Saudi Arabia",
    "Belgium",
    "Croatia",
    "Denmark",
    "England",
    "France",
    "Germany",
    "Iceland",
    "Poland",
    "Portugal",
    "Russia",
    "Serbia",
    "Spain",
    "Sweden",
    "Switzerland",
    "Costa Rica",
    "Mexico",
    "Panama",
    "Argentina",
    "Brazil",
    "Colombia",
    "Peru",
    "Uruguay"
]

team_names = list(data["Home Team Name"].unique()) + list(data["Away Team Name"].unique()) + word_cup_teams

In [8]:
team_name_encoder = LabelEncoder().fit(team_names)

In [9]:
X["Home Team Name"] = team_name_encoder.transform(X["Home Team Name"])
X["Away Team Name"] = team_name_encoder.transform(X["Away Team Name"])

### Feature Selection

In [10]:
feature_names = []

COLUMNS = []
for column_name in X.columns:
    for feature_name in feature_names:
        if feature_name in column_name:
            COLUMNS.append(column_name)
            break

COLUMNS = [
    'Away Team Name',
    'Home Team Name',

    'home_rank',
    'home_cur_year_avg',
    
    'away_rank',
    'away_cur_year_avg',
    
    'Home Avg Goals',
    'Away Avg Goals'
]
X = X[COLUMNS]

In [11]:
COLUMNS

['Away Team Name',
 'Home Team Name',
 'home_rank',
 'home_cur_year_avg',
 'away_rank',
 'away_cur_year_avg',
 'Home Avg Goals',
 'Away Avg Goals']

# Traning Session

In [12]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 8 columns):
Away Team Name       306 non-null int64
Home Team Name       306 non-null int64
home_rank            306 non-null int64
home_cur_year_avg    306 non-null float64
away_rank            306 non-null int64
away_cur_year_avg    306 non-null float64
Home Avg Goals       306 non-null float64
Away Avg Goals       306 non-null float64
dtypes: float64(4), int64(4)
memory usage: 19.2 KB


In [13]:
X.describe()

,Away Team Name,Home Team Name,home_rank,home_cur_year_avg,away_rank,away_cur_year_avg,Home Avg Goals,Away Avg Goals
count,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000
mean,14.980392,13.859477,23.506536,503.409379,25.362745,501.238758,0.180065,0.134804
std,9.928578,9.794438,20.652669,188.348007,21.444915,180.352170,0.530661,0.469572
min,0.000000,0.000000,1.000000,47.490000,1.000000,47.490000,0.000000,0.000000
25%,5.000000,5.000000,7.000000,374.012500,7.000000,369.950000,0.000000,0.000000
50%,16.000000,11.000000,17.000000,504.430000,18.000000,473.830000,0.000000,0.000000
75%,23.000000,23.000000,37.000000,631.380000,41.000000,625.970000,0.000000,0.000000
max,32.000000,32.000000,100.000000,1090.540000,102.000000,1090.540000,3.000000,4.000000


In [14]:
model = SVC(C=1.0, kernel="rbf", gamma="auto", probability=True)

In [15]:
model.fit(X, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

# Save model and encoders

In [16]:
with open("svm_model.b", "wb") as f:
    pickle.dump(model, f)
    
with open("team_name_encoder.b", "wb") as f:
    pickle.dump(team_name_encoder, f)